# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 設定 data_path
dir_data = './data/'

In [2]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: ./data/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


In [4]:
# 如果欄位中有 NA, describe 會有問題
app_train['AMT_ANNUITY'].describe()

count    307499.000000
mean      27108.573909
std       14493.737315
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64

In [5]:
app_train['AMT_ANNUITY'].value_counts()

9000.0     6385
13500.0    5514
6750.0     2279
10125.0    2035
37800.0    1602
11250.0    1459
26217.0    1453
20250.0    1345
12375.0    1339
31653.0    1269
26640.0    1236
22500.0    1234
16875.0    1213
36459.0    1191
33025.5     938
26446.5     916
22018.5     904
23773.5     875
25537.5     836
26316.0     829
31261.5     822
21906.0     800
7875.0      741
40320.0     703
21888.0     688
32602.5     687
15750.0     684
27000.0     681
19867.5     653
33750.0     652
           ... 
71910.0       1
72949.5       1
36472.5       1
81202.5       1
40603.5       1
47205.0       1
39087.0       1
81225.0       1
49932.0       1
73003.5       1
53505.0       1
58243.5       1
73053.0       1
81049.5       1
62703.0       1
45877.5       1
56470.5       1
62383.5       1
63499.5       1
81094.5       1
73048.5       1
91732.5       1
56538.0       1
54297.0       1
81108.0       1
15210.0       1
50265.0       1
73012.5       1
40558.5       1
4437.0        1
Name: AMT_ANNUITY, Lengt

In [36]:
"""
YOUR CODE HERE
"""
# 1: 計算 AMT_ANNUITY 的 q0 - q100
q_all = [np.percentile(app_train['AMT_ANNUITY'][~app_train['AMT_ANNUITY'].isnull()], q=i) for i in range(101)]

pd.DataFrame({'q': list(range(101)),
              'value': q_all})

,q,value
0,0,1615.50
1,1,6182.91
2,2,6750.00
3,3,7875.00
4,4,8703.00
5,5,9000.00
6,6,9000.00
7,7,9553.50
8,8,10125.00
9,9,10503.00


In [61]:
#q_all[0:51] 
q_all[50]  #24903.0

24903.0

In [62]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

print('-----------------------------------------')
print(app_train['AMT_ANNUITY'][app_train['AMT_ANNUITY'].isnull()])

print('-----------------------------------------')
app_train.loc[app_train['AMT_ANNUITY'].isnull(),'SK_ID_CURR':'AMT_ANNUITY']

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
-----------------------------------------
47531    NaN
50035    NaN
51594    NaN
55025    NaN
59934    NaN
75873    NaN
89343    NaN
123872   NaN
207186   NaN
227939   NaN
239329   NaN
241835   NaN
Name: AMT_ANNUITY, dtype: float64
-----------------------------------------


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY
47531,155054,0,Cash loans,M,N,N,0,180000.0,450000.0,NaN
50035,157917,0,Cash loans,F,N,N,0,94500.0,450000.0,NaN
51594,159744,0,Cash loans,F,N,N,0,202500.0,539100.0,NaN
55025,163757,0,Cash loans,F,N,N,0,162000.0,296280.0,NaN
59934,169487,0,Cash loans,M,Y,N,0,202500.0,360000.0,NaN
75873,187985,0,Cash loans,M,Y,N,0,144000.0,219249.0,NaN
89343,203726,0,Cash loans,F,Y,N,0,90000.0,157500.0,NaN
123872,243648,0,Cash loans,F,N,Y,0,202500.0,929088.0,NaN
207186,340147,0,Cash loans,M,N,N,0,171000.0,486000.0,NaN
227939,364022,0,Cash loans,F,N,Y,0,315000.0,628069.5,NaN


In [63]:
# 2.1 將 NAs 以 q50 填補
print("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

"""
Your Code Here
"""
q_50 = np.percentile(app_train['AMT_ANNUITY'][~app_train['AMT_ANNUITY'].isnull()], q=50)
print("the q_50 is: %f" % q_50)
app_train.loc[app_train['AMT_ANNUITY'].isnull(),'AMT_ANNUITY'] = q_50

print("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train['AMT_ANNUITY'].isnull()))

Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12
the q_50 is: 24903.000000
After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [64]:
# 2.2 Normalize values to -1 to 1
print("== Original data range ==")
print(app_train['AMT_ANNUITY'].describe())

def normalize_value(x):
    """
    Your Code Here, compelete this function
    """
    x = ((x - np.min(x)) / (np.max(x) - np.min(x)) - 0.5) * 2
    return x        

app_train['AMT_ANNUITY_NORMALIZED'] = normalize_value(app_train['AMT_ANNUITY'])

print("== Normalized data range ==")
app_train['AMT_ANNUITY_NORMALIZED'].describe()

== Original data range ==
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
== Normalized data range ==


count    307511.000000
mean         -0.801154
std           0.113049
min          -1.000000
25%          -0.883714
50%          -0.818357
75%          -0.742752
max           1.000000
Name: AMT_ANNUITY_NORMALIZED, dtype: float64

In [74]:
print("the app_train max is: %f" % app_train['AMT_ANNUITY'].max())
print("the app_train min is: %f" % app_train['AMT_ANNUITY'].min())

###似乎會發生小數點誤差
print("the min normalize_value is: %f" % 
      (2*((app_train['AMT_ANNUITY'].min()-app_train['AMT_ANNUITY'].min()/app_train['AMT_ANNUITY'].max()-app_train['AMT_ANNUITY'].min())-0.5)))

the app_train max is: 258025.500000
the app_train min is: 1615.500000
the min normalize_value is: -1.012522


In [88]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %f" % sum(app_train['AMT_GOODS_PRICE'].isnull()))
print('-----------------------------------------')
print('-----------------------------------------')
print(app_train['AMT_GOODS_PRICE'][app_train['AMT_GOODS_PRICE'].isnull()])
print('-----------------------------------------')
print('-----------------------------------------')
app_train.loc[app_train['AMT_GOODS_PRICE'].isnull(),'SK_ID_CURR':'AMT_GOODS_PRICE']


Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278.000000
-----------------------------------------
-----------------------------------------
724      NaN
5937     NaN
6425     NaN
6703     NaN
7647     NaN
7880     NaN
7995     NaN
10819    NaN
11287    NaN
13008    NaN
14699    NaN
15953    NaN
18935    NaN
19178    NaN
19921    NaN
21193    NaN
21338    NaN
21546    NaN
25391    NaN
26398    NaN
26736    NaN
27003    NaN
28201    NaN
29059    NaN
30164    NaN
30294    NaN
32143    NaN
36250    NaN
36721    NaN
39230    NaN
          ..
278747   NaN
279583   NaN
279997   NaN
281050   NaN
281653   NaN
282002   NaN
282270   NaN
282847   NaN
283159   NaN
285269   NaN
287092   NaN
287347   NaN
289793   NaN
291372   NaN
291384   NaN
292040   NaN
292120   NaN
294136   NaN
295295   NaN
298004   NaN
300107   NaN
302923   NaN
303603   NaN
303621   NaN
304621   NaN
304678   NaN
304784   NaN
305833   NaN
306126   NaN
306273   NaN
Name: AMT_GOODS_PRICE, Length: 278, dtype: float

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE
724,100837,0,Revolving loans,F,N,Y,2,45000.0,135000.0,6750.0,NaN
5937,106955,0,Revolving loans,F,N,N,0,157500.0,450000.0,22500.0,NaN
6425,107494,0,Revolving loans,F,N,N,0,67500.0,202500.0,10125.0,NaN
6703,107822,0,Revolving loans,F,N,N,1,121500.0,180000.0,9000.0,NaN
7647,108913,0,Revolving loans,M,N,Y,0,180000.0,450000.0,22500.0,NaN
7880,109190,1,Revolving loans,F,N,N,0,121500.0,270000.0,13500.0,NaN
7995,109322,0,Revolving loans,M,N,Y,0,112500.0,180000.0,9000.0,NaN
10819,112595,0,Revolving loans,F,N,Y,0,90000.0,270000.0,13500.0,NaN
11287,113148,0,Revolving loans,F,N,Y,4,225000.0,135000.0,6750.0,NaN
13008,115162,0,Revolving loans,F,N,Y,1,157500.0,450000.0,22500.0,NaN


In [96]:
# 3
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

# 列出重複最多的數值
"""
Your Code Here
"""
d = dict()
for i in app_train['AMT_GOODS_PRICE'][~app_train['AMT_GOODS_PRICE'].isnull()]:
    if i in d:
        d[i] += 1
    else:
        d[i] = 1

value_most = max(zip(d.values(), d.keys()))[0]
print(value_most)

mode_goods_price = list(app_train['AMT_GOODS_PRICE'].value_counts().index)
app_train.loc[app_train['AMT_GOODS_PRICE'].isnull(), 'AMT_GOODS_PRICE'] = mode_goods_price[0]

print("After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train['AMT_GOODS_PRICE'].isnull()))

Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278
26022
After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 0


In [95]:
list(app_train['AMT_GOODS_PRICE'].value_counts())[0]

26022